In [1]:
import gradio as gr
import tensorflow as tf
import cv2
import numpy as np
from PIL import Image 

In [18]:
model = tf.keras.models.load_model('new_model.keras')

In [19]:
class_names = ["daisy", "dandelion", "rose", "sunflower", "tulip"]

In [20]:
# Hàm xử lý từng khung hình và nhận dạng hoa
def process_frame(frame):
    # Thay đổi kích thước khung hình về kích thước mô hình yêu cầu (64x64) với interpolation hiệu quả
    resized_frame = cv2.resize(frame, (64, 64), interpolation=cv2.INTER_AREA)
    # Chuẩn hóa khung hình từ [0, 255] về [0, 1]
    normalized_frame = resized_frame.astype(np.float32) / 255.0
    # Thêm chiều batch để phù hợp với input_shape của mô hình
    expanded_frame = np.expand_dims(normalized_frame, axis=0)
    # Dự đoán nhãn
    predictions = model.predict(expanded_frame)
    predicted_class = np.argmax(predictions[0])
    # Trả về tên loại hoa
    return class_names[predicted_class]

In [21]:
# Hàm nhận dạng hoa trong video và trả về danh sách thời gian và nhãn hoa
def recognize_flowers_in_video(video_file):
    cap = cv2.VideoCapture(video_file.name)
    results = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Lấy thời gian hiện tại trong video
        time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0  # thời gian tính bằng giây
        # Nhận dạng hoa trên khung hình
        flower_name = process_frame(frame)
        # Lưu kết quả với thời gian tương ứng
        results.append((time, flower_name))

    cap.release()
    return results

In [22]:
# Hàm nhận dạng hoa từ ảnh
def recognize_flowers_in_image(image_file):
    results = []
    # Mở ảnh từ tệp bằng PIL
    image = Image.open(image_file.name).convert('RGB')
    # Chuyển đổi ảnh thành mảng NumPy
    image_np = np.array(image)
    # Chuyển đổi ảnh về kích thước mà mô hình yêu cầu (64x64)
    resized_image = cv2.resize(image_np, (64, 64))
    # Chuyển đổi ảnh về định dạng mà mô hình yêu cầu
    normalized_image = resized_image / 255.0
    # Mở rộng chiều để phù hợp với input_shape của mô hình
    expanded_image = np.expand_dims(normalized_image, axis=0)
    # Dự đoán nhãn
    predictions = model.predict(expanded_image)
    predicted_class = np.argmax(predictions, axis=1)[0]
    flower_name = class_names[predicted_class]
    # Thêm kết quả vào danh sách
    results.append((0, flower_name))  # Thời gian là 0 cho ảnh
    return results

In [23]:
def gradio_processing(file):
    # Kiểm tra kiểu đầu vào và gọi hàm xử lý tương ứng
    if file.name.endswith('.mp4') or file.name.endswith('.avi'):
        results = recognize_flowers_in_video(file)
    else:
        results = recognize_flowers_in_image(file)
    
    # Tạo danh sách hiển thị kết quả với tên loại hoa
    display_results = [{"Time (s)": time, "Flower Name": flower_name} for time, flower_name in results]
    return display_results

In [24]:
# Tạo giao diện Gradio
interface = gr.Interface(
    fn=gradio_processing,
    inputs=gr.File(label="Upload File"),
    outputs=gr.JSON(),
    title="Flower Recognition",
    description="Upload an image or video to recognize flowers and get the results with timestamps."
)

In [25]:
# Chạy ứng dụng Gradio
interface.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━